In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.figure_factory as ff
import seaborn as sns
import numpy as np
import string

# Removing stop words like 'the', 'a', 'an', 'in'
from nltk.corpus import stopwords

# Return the ngrams generated from a sequence of items, as an iterator
from nltk.util import ngrams

# Convert a collection of text documents to a matrix of token counts
from sklearn.feature_extraction.text import CountVectorizer
from  sklearn.metrics  import accuracy_score, confusion_matrix
from plotly.subplots import make_subplots

from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))
import re
import nltk
from nltk.tokenize import word_tokenize
import gensim
import string
from tqdm import tqdm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

# Machine Learning
from sklearn.naive_bayes import MultinomialNB # good one
from sklearn.linear_model import LogisticRegression # best
from lightgbm import LGBMClassifier

In [ ]:
tweet = pd.read_csv('D:/Python/Anaconda/Kaggle/Working/Natural Language Processing with Disaster Tweets/nlp-getting-started/train.csv')
test = pd.read_csv('D:/Python/Anaconda/Kaggle/Working/Natural Language Processing with Disaster Tweets/nlp-getting-started/test.csv')

In [ ]:
tweet.head(3)

In [ ]:
print('There are {} rows and {} columns in train'.format(tweet.shape[0],tweet.shape[1]))
print('There are {} rows and {} columns in train'.format(test.shape[0],test.shape[1]))

In [ ]:
fig = go.Figure()

to_plot = tweet['target'].replace({0: 'No disaster', 1: 'Disaster'}).value_counts()
labels = to_plot.index
values = to_plot.values

fig.add_trace(go.Pie(
    labels = labels,
    values = values,
    textinfo='label+percent'
))

fig.update_layout(
    title_text='Target balance',
    template='plotly_dark'
)

<h1>Number of characters in tweets</h1>

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=('Disaster tweets', 'No disaster tweets'))

tweet_len_1 = tweet[tweet['target']==1]['text'].str.len()

fig.add_trace(
    go.Histogram(x = tweet_len_1, name='Disaster tweets'),
    row=1,
    col=1
    )


tweet_len_0 = tweet[tweet['target']==0]['text'].str.len()

fig.add_trace(
    go.Histogram(x = tweet_len_0, name='No disaster tweets'),
    row=1,
    col=2
    )

fig.update_layout(
    showlegend=False,
    title_text='Characters in tweets',
    template='plotly_dark'
)

<h1>Number of words in tweets</h1>

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=('Disaster tweets', 'No disaster tweets'))

tweet_len_1 = tweet[tweet['target']==1]['text'].str.split().map(lambda x: len(x))

fig.add_trace(
    go.Histogram(x = tweet_len_1, name='Disaster tweets'),
    row=1,
    col=1
    )


tweet_len_0 = tweet[tweet['target']==0]['text'].str.split().map(lambda x: len(x))

fig.add_trace(
    go.Histogram(x = tweet_len_0, name='No disaster tweets'),
    row=1,
    col=2
    )

fig.update_layout(
    showlegend=False,
    title_text='Characters in words',
    template='plotly_dark'
)

<h1>Average word length in a tweets</h1>

In [ ]:
word_dis = tweet[tweet['target']==1]['text'].str.split().apply(lambda x: [len(i) for i in x])
word_ndis = tweet[tweet['target']==0]['text'].str.split().apply(lambda x: [len(i) for i in x])

fig = ff.create_distplot(
    [word_dis.map(lambda x: np.mean(x)), word_ndis.map(lambda x: np.mean(x))],
    ['Disaster', 'No disaster'],
    bin_size=0.2,
    colors=['#ff5050', '#0066ff']
)

fig.update_layout(
    title_text = 'Average word length in each tweet',
    template='plotly_dark'
)

fig.show()

<h1>Common stopwords in tweets</h1>

In [ ]:
def words_list(target):
    words = []

    for x in tweet[tweet['target']==target]['text'].str.split():
        for i in x:
            words.append(i)
    return words

In [ ]:
words_list_0 = words_list(0)
dic = defaultdict(int)

for word in words_list_0:
    if word in stop:
        dic[word] += 1
        
top=sorted(dic.items(), key=lambda x:x[1], reverse=True)[:10] 

In [ ]:
fig = go.Figure()

x, y = zip(*top)

fig.add_trace(go.Bar(
    x = x,
    y = y
))

fig.update_layout(
    title_text='Common stopwords in tweets with class 0',
    template='plotly_dark'
)

In [ ]:
words_list_1 = words_list(1)
dic = defaultdict(int)

for word in words_list_0:
    if word in stop:
        dic[word] += 1
        
top=sorted(dic.items(), key=lambda x:x[1], reverse=True)[:10] 

In [ ]:
fig = go.Figure()

x, y = zip(*top)

fig.add_trace(go.Bar(
    x = x,
    y = y
))

fig.update_layout(
    title_text='Common stopwords in tweets with class 1',
    template='plotly_dark'
)

<h1>Analyzing punctuations</h1>

In [ ]:
punctuations = words_list(0)

dic = defaultdict(int)
special = string.punctuation
for i in punctuations:
    if i in special:
        dic[i] += 1

x, y = zip(*dic.items())

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x = x,
    y = y
))

fig.update_layout(
    title_text = 'Punctuations for class 0',
    template='plotly_dark'
)

In [ ]:
punctuations = words_list(1)

dic = defaultdict(int)
special = string.punctuation
for i in punctuations:
    if i in special:
        dic[i] += 1

x, y = zip(*dic.items())

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x = x,
    y = y
))

fig.update_layout(
    title_text = 'Punctuations for class 1',
    template='plotly_dark'
)

<h1>Common words</h1>

In [ ]:
counter = Counter(words_list(1))
most = counter.most_common()
x = list()
y = list()

for word, count in most[:40]:
    if word not in stop:
        x.append(word)
        y.append(count)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x = x,
    y = y
))

fig.update_layout(
    title_text = 'Common words for class 1',
    template='plotly_dark'
)

In [ ]:
counter = Counter(words_list(0))
most = counter.most_common()
x = list()
y = list()

for word, count in most[:40]:
    if word not in stop:
        x.append(word)
        y.append(count)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x = x,
    y = y
))

fig.update_layout(
    title_text = 'Common words for class 0',
    template='plotly_dark'
)

<h1>Ngram analysis</h1>

In [ ]:
def get_top_tweet_bigrams(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2,2)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
fig = go.Figure()

top_tweet_bigrams = get_top_tweet_bigrams(tweet['text'])[:10]
x, y = map(list, zip(*top_tweet_bigrams))

fig.add_trace(go.Bar(
    x = x,
    y = y
))

fig.update_layout(
    title_text = 'Bigrams',
    template='plotly_dark'
)

<h1> Data Cleaning </h1>

In [ ]:
df = pd.concat([tweet, test])
df.shape

<h1> Removing urls </h1>

In [ ]:
test = "New competition launched :https://www.kaggle.com/c/nlp-getting-started"

In [ ]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

In [ ]:
remove_URL(test)

In [ ]:
# Delete url from all rows
df['text'] = df['text'].apply(lambda x: remove_URL(x))

<h1>Removing HTML tags</h1>

In [ ]:
test = """<div>
<h1>Real or Fake</h1>
<p>Kaggle </p>
<a href="https://www.kaggle.com/c/nlp-getting-started">getting started</a>
</div>"""

In [ ]:
def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)

In [ ]:
print(remove_html(test))

In [ ]:
# Delete HTML from all rows
df['text'] = df['text'].apply(lambda x: remove_html(x))

<h1>Removing Emojis</h1>

In [ ]:
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

remove_emoji("Omg another Earthquake 😔😔")

In [ ]:
# Delete emojis from all rows
df['text'] = df['text'].apply(lambda x: remove_emoji(x))

<h1>Removing punctuations</h1>

In [ ]:
def remove_punctuations(text):
    """This function creates a dictionary mapping of every character from string.punctuation to None"""
    table = str.maketrans('','', string.punctuation)
    return text.translate(table)

In [ ]:
example = "I am a #king"
print(remove_punctuations(example))

In [ ]:
# Delete punctuations from all rows
df['text'] = df['text'].apply(lambda x: remove_punctuations(x))

In [ ]:
df

<h1> Vectorization </h1>

In [ ]:
data_train = df[:7613]
data_test = df[7613:]

In [ ]:
X = data_train['text']
y = data_train['target']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(X_train) # Vectors
val_vectors = vectorizer.transform(X_val)
test_vectors = vectorizer.transform(data_test['text'])

print(train_vectors.shape, val_vectors.shape)

In [ ]:
# Convert vectorized and splited data into one dataframe to make evaluation simpler
to_model_evaluation = pd.concat([pd.DataFrame(train_vectors.toarray()), pd.DataFrame(val_vectors.toarray())])
y_all = pd.concat([y_train, y_val])

In [ ]:
def model_evaluation(model):
    # Train our model
    model.fit(train_vectors, y_train)
    y_pred = model.predict(val_vectors)

    # F1 score
    scores_f1 = cross_val_score(model, to_model_evaluation, y_all, scoring = 'f1' , cv=4)
    print('F1 scores: {}'.format(scores_f1))
    print('F1 mean score: {}'.format(scores_f1.mean()))

    # Accuracy score
    scores_acc = cross_val_score(model, to_model_evaluation, y_all, cv=4)
    print('Acc scores: {}'.format(scores_acc))
    print('Acc mean score: {}'.format(scores_acc.mean()))
    
    # Confusion matrix
    print('Confusion Matrix: ')
    matrix = confusion_matrix(y_val, y_pred)
    group_names = ['True Negative','False Positive','False Negative','True Positive']
    group_counts =['{0:0.0f}'.format(value) for value in matrix.flatten()]
    group_percentages = ['{0:.2%}'.format(value) for value in matrix.flatten()/np.sum(matrix)]
    
    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2, 2)

    sns.heatmap(matrix, annot=labels, fmt='', cmap='rocket_r')

<h1> Model Creation </h1>

In [ ]:
# Naive Bayes Classification
NB_model = MultinomialNB()

# LogisticRegression
LR_model = LogisticRegression()

# LGBMClassifier
LGM_model = LGBMClassifier()

In [ ]:
model_evaluation(NB_model)

In [ ]:
model_evaluation(LR_model)

In [ ]:
model_evaluation(LGM_model)

In [ ]:
sample_sub = pd.read_csv('D:/Python/Anaconda/Kaggle/Working/Natural Language Processing with Disaster Tweets/nlp-getting-started/sample_submission.csv')

In [ ]:
y_pre=LR_model.predict(test_vectors)
y_pre=np.round(y_pre).astype(int).reshape(3263)
sub=pd.DataFrame({'id':sample_sub['id'].values.tolist(),'target':y_pre})
sub.to_csv('submission2.csv',index=False)